In [564]:
# import necessary modules
import pandas as pd
import numpy as np

# load the necessary csv file
csv_fp = 'purchase_data.csv'
# allows the data to be read from the desired file
purch_data = pd.read_csv(csv_fp)

In [565]:
# counts unique instances of purchases based on customer SN
tot_peps = purch_data['SN'].unique()

# displays total number of players
tot_peps_num = len(tot_peps)
tot_players = [{'Total Players': tot_peps_num}]
tot_pepsdf = pd.DataFrame(tot_players)
tot_pepsdf

,Total Players
0,576


In [566]:
# finds the unqiue instances of items, average sales price, total number of transactions, and total revenue
poss_item = purch_data['Item Name'].unique()

# uses len() to find the total number of possible item choices and mean() to find the average
# round() is used to allow for a number that is tailored for the circumstance, which in this case is to the second digit
num_items = len(poss_item)
avg_price = purch_data['Price'].mean()
avg_price = round(avg_price, 2)
tot_ampurch = len(purch_data)
tot_rev = purch_data['Price'].sum()

# creates layout of desired output as well as making it into a dataframe
totals = [{'Unique Items': num_items, 'Average Price': avg_price, 'Number of Transactions': tot_ampurch, 'Total Revenue':
          tot_rev}]
totals = pd.DataFrame(totals)
totals

,Average Price,Number of Transactions,Total Revenue,Unique Items
0,3.05,780,2379.77,179


In [567]:
# drops duplicates in set using the "SN" column as the key for determining the duplicates, only the first instance of the 
# duplicate will be kept and if their is a duplicate it will not replace the value of its original. 
g_data = purch_data.drop_duplicates(subset ='SN', keep ='first', inplace = False)

# counts the number of times a value shows up in the Gender column
g_data = g_data['Gender'].value_counts()
g_tot = g_data['Male'] + g_data['Female'] + g_data['Other / Non-Disclosed']
g_df = {'Gender': ['Male', 'Female', 'Other / Non-Disclosed'], 'Totals': [g_data['Male'], g_data['Female'], g_data['Other / Non-Disclosed']], 'Percent': [round((g_data['Male']/g_tot)*100, 2), round((g_data['Female']/g_tot)*100, 2), round((g_data['Other / Non-Disclosed']/g_tot)*100, 2)]}

# creates g_df into a dataframe and sets the index to the genders
g_df = pd.DataFrame(g_df)
g_df.set_index('Gender')

,Totals,Percent
Gender,,
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91


In [568]:
# finds data needed for individuals sales based on gender except for average per sales per person
p_count = purch_data['Gender'].value_counts()
purch_data2 = purch_data.set_index('Gender')
mavg = round(purch_data2.loc['Male', 'Price'].mean(), 2)
m_tot = purch_data2.loc['Male', 'Price'].sum()
favg = round(purch_data2.loc['Female', 'Price'].mean(), 2)
f_tot = purch_data2.loc['Female', 'Price'].sum()
ondavg = round(purch_data2.loc['Other / Non-Disclosed', 'Price'].mean(), 2)
ond_tot = purch_data2.loc['Other / Non-Disclosed', 'Price'].sum()

# finds the names for each gender
def gender_sn(df, gender):
    df = pd.DataFrame(df.set_index('Gender'))
    df = df.loc[str(gender)]
    names = df['SN'].unique()
    return names

# utilizes function to find names for each gender so we can use the names for finding the avg. purch/person
m_name = gender_sn(purch_data, 'Male')
f_name = gender_sn(purch_data, 'Female')
ondp_name = gender_sn(purch_data, 'Other / Non-Disclosed')
purch_data = purch_data.set_index('SN')

In [569]:
# function to find purch/person
def per_peps(names, df):
    df = pd.DataFrame(df)
    names_avg = [df.loc[str(x), 'Price'].sum() for x in names]
    return round(sum(names_avg)/len(names), 2)

mp_avg = per_peps(m_name, purch_data)
fp_avg = per_peps(f_name, purch_data)
ondp_avg = per_peps(ondp_name, purch_data)

# creates df for informatioin
ginfo_df = {'Gender': ['Male', 'Female', 'Other / Non-Disclosed'], 'Total Revenue': [m_tot, f_tot, ond_tot], 
            'Average Purchase Price': [mavg, favg, ondavg], 'Number of Transactions': [p_count['Male'], p_count['Female'], p_count['Other / Non-Disclosed']], 
           'Average Spent/Buyer': [mp_avg, fp_avg, ondp_avg]}
ginfo_df = pd.DataFrame(ginfo_df)
ginfo_df = ginfo_df.set_index('Gender')
ginfo_df

,Total Revenue,Average Purchase Price,Number of Transactions,Average Spent/Buyer
Gender,,,,
Male,1967.64,3.02,652,4.07
Female,361.94,3.20,113,4.47
Other / Non-Disclosed,50.19,3.35,15,4.56


In [570]:
# creates dictionary with empty values for purpose of making areas for so to be generated values
age_setup = {'>10': [], '10 - 14': [], '15 - 19': [], '20 - 24': [], '25 - 29': [], '30 - 34': [], '35 - 40': [], '40+': []}
purch_data = purch_data.groupby('SN')

In [571]:
# creates set of all ages without repeating SN values which are used as key to get ages
age_set = purch_data['Age'].describe().values
ages = [x[-1] for x in age_set]

# iteration for giving age_setup values based on number of instances for each age
nn = 0
n = 9
for x in age_setup:
    for y in ages:
        if nn < y < n + 1:
            age_setup[x].append(1)
            next
        else:
            next
    age_setup[x] = [len(age_setup[x])]
    n = n + 5
    nn = n - 5
    age_setup[x].append(round(age_setup[x][0]/len(ages)*100, 2))
    next

In [582]:
# creates df for ages (found the '.T' online and it said it was an easy way to switch axis for indxes)
age_df = pd.DataFrame(age_setup).T
age_df

,0,1
>10,17.0,2.95
10 - 14,22.0,3.82
15 - 19,107.0,18.58
20 - 24,258.0,44.79
25 - 29,77.0,13.37
30 - 34,52.0,9.03
35 - 40,31.0,5.38
40+,11.0,1.91
